## 📦 Ячейка 1 — подключение

In [1]:
import pandahouse
import pandas as pd

connection = {
    "host": "http://clickhouse:8123",
    "database": "churn"
}


## 📊 Ячейка 2 — посмотреть сырые события

In [2]:
events_df = pandahouse.read_clickhouse(
    query="SELECT * FROM churn.events LIMIT 100",
    connection=connection
)

events_df

,user_id,event_time,event_type,session_id
0,1,2025-12-29 16:38:14,click,35090
1,1,2025-12-29 16:38:14,click,85046
2,1,2025-12-29 16:38:14,click,17160
3,1,2025-12-29 16:38:14,click,61042
4,1,2025-12-29 16:38:14,click,72391
...,...,...,...,...
95,1,2025-12-29 18:53:49,purchase,6002
96,1,2025-12-29 18:53:49,click,21907
97,1,2025-12-29 18:53:49,click,53116
98,1,2025-12-29 18:53:49,purchase,31583


## 🧠 Ячейка 3 — посмотреть фичи

In [18]:
features_df = pandahouse.read_clickhouse(
    query="SELECT * FROM churn.user_features",
    connection=connection
)

features_df.describe()


,user_id,days_active,total_events,avg_events_per_day,last_event_days_ago
count,500.000000,500.0,500.000000,500.000000,500.0
mean,250.500000,2.0,158.420000,79.209999,0.0
std,144.481833,0.0,27.072818,13.536409,0.0
min,1.000000,2.0,82.000000,41.000000,0.0
25%,125.750000,2.0,141.000000,70.500000,0.0
50%,250.500000,2.0,158.000000,79.000000,0.0
75%,375.250000,2.0,177.000000,88.500000,0.0
max,500.000000,2.0,244.000000,122.000000,0.0


## 🔍 Ячейка 4 — sanity checks

In [4]:
assert features_df.isnull().sum().sum() == 0
assert (features_df["days_active"] > 0).all()


In [5]:
import mlflow

mlflow.set_tracking_uri("http://mlflow:5000")

In [12]:
!pip install dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 870.6 kB/s eta 0:00:00a 0:00:01


## Как посмотреть версии программно

In [14]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

for mv in client.search_model_versions("name='churn_model'"):
    print(
        f"version={mv.version}, "
        f"stage={mv.current_stage}, "
        f"run_id={mv.run_id}"
    )


version=2, stage=Production, run_id=6357920c53394259922e42e421c22726
version=1, stage=None, run_id=981ea63c41f0473a93f2755396efb2eb


## Загрузить последнюю модель

In [17]:
import mlflow.pyfunc
from tqdm import tqdm

model = mlflow.pyfunc.load_model(
    model_uri="models:/churn_model/latest"
)

2025/12/30 16:50:21 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 3.8.1, required: mlflow==2.9.2)
 - numpy (current: 2.2.6, required: numpy==1.26.4)
 - packaging (current: 23.1, required: packaging==23.2)
 - pandas (current: 2.3.3, required: pandas==2.1.4)
 - psutil (current: 5.9.5, required: psutil==6.0.0)
 - pyyaml (current: 6.0, required: pyyaml==6.0.1)
 - scikit-learn (current: 1.7.2, required: scikit-learn==1.6.1)
 - scipy (current: 1.15.3, required: scipy==1.13.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/12/30 16:50:21 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.9.19`, differs from the version of Python that is currently running, `Python 3.10.11`, and may be incompatible
/opt/conda/lib/pytho

In [8]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 6357920c53394259922e42e421c22726

In [9]:
model = mlflow.pyfunc.load_model(
    model_uri="models:/churn_model/2"
)

2025/12/29 18:59:34 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 3.8.1, required: mlflow==2.9.2)
 - numpy (current: 2.2.6, required: numpy==1.26.4)
 - packaging (current: 23.1, required: packaging==23.2)
 - pandas (current: 2.3.3, required: pandas==2.1.4)
 - psutil (current: 5.9.5, required: psutil==6.0.0)
 - pyyaml (current: 6.0, required: pyyaml==6.0.1)
 - scikit-learn (current: 1.7.2, required: scikit-learn==1.6.1)
 - scipy (current: 1.15.3, required: scipy==1.13.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/12/29 18:59:34 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.9.19`, differs from the version of Python that is currently running, `Python 3.10.11`, and may be incompatible
/opt/conda/lib/pytho

## Перевести модель в Production

In [12]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

client.transition_model_version_stage(
    name="churn_model",
    version=2,
    stage="Production"
)

/tmp/ipykernel_12801/2409283935.py:5: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1767034437136, current_stage='Production', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1767034915097, metrics=None, model_id=None, name='churn_model', params=None, run_id='6357920c53394259922e42e421c22726', run_link='', source='s3://mlflow-artifacts/1/6357920c53394259922e42e421c22726/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='2'>

## Загрузить Production-модель

In [16]:
model = mlflow.pyfunc.load_model(
    model_uri="models:/churn_model/Production"
)

2025/12/30 16:43:33 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 3.8.1, required: mlflow==2.9.2)
 - numpy (current: 2.2.6, required: numpy==1.26.4)
 - packaging (current: 23.1, required: packaging==23.2)
 - pandas (current: 2.3.3, required: pandas==2.1.4)
 - psutil (current: 5.9.5, required: psutil==6.0.0)
 - pyyaml (current: 6.0, required: pyyaml==6.0.1)
 - scikit-learn (current: 1.7.2, required: scikit-learn==1.6.1)
 - scipy (current: 1.15.3, required: scipy==1.13.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/12/30 16:43:34 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.9.19`, differs from the version of Python that is currently running, `Python 3.10.11`, and may be incompatible
/opt/conda/lib/pytho

## предсказания

In [16]:
import pandas as pd

X = pd.DataFrame([
    {
        "days_active": 10,
        "total_events": 42,
        "avg_events_per_day": 4.2,
        "last_event_days_ago": 3
    }
])

pred = model.predict(X)
pred

array([0])

In [17]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 6bd6b6c0d9414f01a0b5eaa782345dfb

In [18]:
pred

array([0])